# **Machine Learning model for gas production prediction**
# **Part 3: Machine Learning Regression Model Training and Evaluation**
### **Author:** M. Sc. Alberto M. Palacio Bastos - Data Scientist
### **Objective:** Build a model to estimate gas production.
### **Dataset:** http://huy302.github.io/interview_dataset.csv